In [1]:
from requests import get
from bs4 import BeautifulSoup
from dataclasses import dataclass
import json
from typing import TypedDict
import os
import time

In [2]:
class Joueur(TypedDict):
    rank: str
    pays: str
    lien_joueur: str
    nom_joueur: str
    pays_abreviation: str
    age: str
    points: str

In [3]:
current_dir: str = os.getcwd()

parent_dir: str = os.path.dirname(current_dir)

file_path: str = os.path.join(parent_dir, "scraping_top_900_H", "joueurs.json")

# Charger le JSON
with open(file_path, "r") as fichier:
    joueurs:list[dict] = json.load(fichier)
    assert len(joueurs) == 900

In [4]:
print(joueurs[0]['lien_joueur'])

https://www.tennisendirect.net/atp/jannik-sinner/


In [5]:
liens_joueurs = (joueurs[i]['lien_joueur'] for i in range(len(joueurs)))

In [6]:
liens_joueurs

<generator object <genexpr> at 0x00000228B003B850>

# Test de récupération des stats d'un joueur (Sinner)

In [7]:
Sinner = "https://www.tennisendirect.net/atp/jannik-sinner/"

In [8]:
reponse = get(Sinner)
print(Sinner, reponse.status_code)
assert reponse.status_code == 200

https://www.tennisendirect.net/atp/jannik-sinner/ 200


In [9]:
detail_Sinner= BeautifulSoup(reponse.text, features="lxml")

In [10]:
profil = detail_Sinner.find_all("div", attrs={"class": "player_stats"})
assert len(profil) == 1
statistiques = detail_Sinner.find_all("table", attrs={"class": "table_stats"})
assert len(statistiques) == 1
*_, derniers_match = detail_Sinner.find_all("table", attrs={"class": "table_pmatches"})

In [11]:
type(statistiques)

bs4.element.ResultSet

In [12]:
print("""profil: {profil} \n stats: {statistiques} \n matchs: {derniers_match}""".format(
    profil=profil, statistiques=statistiques, derniers_match=derniers_match))


profil: [<div class="player_stats">
            Nom: <b><a href="https://www.tennisendirect.net/atp/jannik-sinner/" title="Jannik Sinner">Jannik Sinner</a></b><br/>            Pays: <b>Italy</b><br/>            Date de naissance: <b>16.08.01, 23 ans</b><br/> <a href="https://www.tennisendirect.net/atp/classement/" title="Position dans le classement">Classement ATP</a>: <b>1</b><br/>            TOP position dans le classement: <b>1</b> (04.11.24, 11330 points)<br/>            Points: <b>11330</b><br/>            Primes: <b>17043434 $</b><br/>            Total de matchs: <b>483</b><br/>            Victoires: <b>348</b><br/>            Taux de réussite: <b>72.05 %</b><br/>
</div>] 
 stats: [<table class="table_stats">
<tr class="header"><td>année</td><td>sommaire</td><td><a href="?su=1" rel="nofollow" title="surface: dure">dure</a></td><td><a href="?su=2" rel="nofollow" title="surface: terre battue">terre battue</a></td><td><a href="?su=3" rel="nofollow" title="surface: salle">salle</a></

In [13]:
@dataclass
class Profil:
    nom: str
    pays: str
    date_naissance: str
    classement_atp: str
    points: str
    primes: str
    total_match: str
    victoires: str
    taux_reussite: str

In [14]:
@dataclass
class Statistiques:
    annee: str
    sommaire: str
    dure: str
    terre_battue: str
    salle: str
    carpet: str
    gazon: str
    acryl: str

In [15]:
@dataclass
class Matchs:
    date: str
    stage: str
    nom_vainqueur: str
    nom_perdant: str
    score: str
    resultat: str
    lien_detail: str
    tournoi: str
    type_terrain :str

## Profil 

In [16]:
print(profil)

[<div class="player_stats">
            Nom: <b><a href="https://www.tennisendirect.net/atp/jannik-sinner/" title="Jannik Sinner">Jannik Sinner</a></b><br/>            Pays: <b>Italy</b><br/>            Date de naissance: <b>16.08.01, 23 ans</b><br/> <a href="https://www.tennisendirect.net/atp/classement/" title="Position dans le classement">Classement ATP</a>: <b>1</b><br/>            TOP position dans le classement: <b>1</b> (04.11.24, 11330 points)<br/>            Points: <b>11330</b><br/>            Primes: <b>17043434 $</b><br/>            Total de matchs: <b>483</b><br/>            Victoires: <b>348</b><br/>            Taux de réussite: <b>72.05 %</b><br/>
</div>]


In [ ]:
def genere_profil(profil):
    div = profil[0]

    try:
        nom_joueur = div.find("a").text.strip()
        pays = div.find_all("b")[1].text.strip()
        naissance = div.find_all("b")[2].text.strip()
        classement_atp = div.find_all("b")[3].text.strip()
        points = div.find_all("b")[5].text.strip()
        primes = div.find_all("b")[6].text.strip()
        total_match = div.find_all("b")[7].text.strip()
        victoires = div.find_all("b")[8].text.strip()
        taux_reussite = div.find_all("b")[9].text.strip()
    
    except (IndexError, AttributeError):
        nom_joueur, pays, naissance, classement_atp, points, primes, total_match, victoires, taux_reussite =(
            "NA",
            "NA",
            "NA",
            "NA",
            "NA",
            "NA",
            "NA",
            "NA",
            "NA",
        )
        
    return Profil(
        nom=nom_joueur,
        pays=pays,
        date_naissance=naissance,
        classement_atp=classement_atp,
        points=points,
        primes=primes,
        total_match=total_match,
        victoires=victoires,
        taux_reussite=taux_reussite,
    )  


In [18]:
Sinner_profil = genere_profil(profil)
Sinner_profil

Profil(nom='Jannik Sinner', pays='Italy', date_naissance='16.08.01, 23 ans', classement_atp='1', points='11330', primes='17043434 $', total_match='483', victoires='348', taux_reussite='72.05 %')

## Statistiques

In [28]:
def extraire_lignes(table):
    return table.find_all(
        "tr", class_=lambda class_name: class_name in ["pair", "unpair"]
    )

In [ ]:
def genere_statistiques(ligne):
    colonnes = [td.text.strip() for td in ligne.find_all("td")]
    
    if len(colonnes) == 8:
        annee, sommaire, dure, terre_battue, salle, carpet, gazon, acryl = colonnes
    else:
        print("Format inattendu dans la ligne:", colonnes)
        return None

    return Statistiques(
        annee = annee,
        sommaire = sommaire,
        dure = dure,
        terre_battue = terre_battue,
        salle = salle,
        carpet = carpet,
        gazon = gazon,
        acryl = acryl,
    ) 

In [25]:
type(statistiques[0])

bs4.element.Tag

In [22]:
print(statistiques)

[<table class="table_stats">
<tr class="header"><td>année</td><td>sommaire</td><td><a href="?su=1" rel="nofollow" title="surface: dure">dure</a></td><td><a href="?su=2" rel="nofollow" title="surface: terre battue">terre battue</a></td><td><a href="?su=3" rel="nofollow" title="surface: salle">salle</a></td><td><a href="?su=4" rel="nofollow" title="surface: carpet">carpet</a></td><td><a href="?su=5" rel="nofollow" title="surface: gazon">gazon</a></td><td><a href="?su=6" rel="nofollow" title="surface: acryl">acryl</a></td></tr><tr class="pair"><td><a href="?y=2024" rel='\"nofollow\"' title="année: 2024">2024</a></td><td>66/7</td><td><a href="?su=1&amp;y=2024" rel="nofollow" title="surface: dure, année: 2024, victoires: 41, défaites: 3, Taux de réussite: 93.18 %">41/3</a></td><td><a href="?su=2&amp;y=2024" rel="nofollow" title="surface: terre battue, année: 2024, victoires: 11, défaites: 3, Taux de réussite: 78.57 %">11/3</a></td><td><a href="?su=3&amp;y=2024" rel="nofollow" title="surface

In [39]:
lignes_stats = extraire_lignes(statistiques[0])
lignes_stats

[<tr class="pair"><td><a href="?y=2024" rel='\"nofollow\"' title="année: 2024">2024</a></td><td>66/7</td><td><a href="?su=1&amp;y=2024" rel="nofollow" title="surface: dure, année: 2024, victoires: 41, défaites: 3, Taux de réussite: 93.18 %">41/3</a></td><td><a href="?su=2&amp;y=2024" rel="nofollow" title="surface: terre battue, année: 2024, victoires: 11, défaites: 3, Taux de réussite: 78.57 %">11/3</a></td><td><a href="?su=3&amp;y=2024" rel="nofollow" title="surface: salle, année: 2024, victoires: 5, défaites: 0, Taux de réussite: 100 %">5/0</a></td><td><a href="?su=4&amp;y=2024" rel="nofollow" title="surface: carpet, année: 2024, victoires: 0, défaites: 0, Taux de réussite: 0 %">0/0</a></td><td><a href="?su=5&amp;y=2024" rel="nofollow" title="surface: gazon, année: 2024, victoires: 9, défaites: 1, Taux de réussite: 90 %">9/1</a></td><td><a href="?su=6&amp;y=2024" rel="nofollow" title="surface: acryl, année: 2024, victoires: 0, défaites: 0, Taux de réussite: 0 %">0/0</a></td></tr>,
 <

In [40]:
Sinner_statistiques = [genere_statistiques(ligne) for ligne in lignes_stats if genere_statistiques(ligne)]

In [41]:
Sinner_statistiques

[Statistiques(annee='2024', sommaire='66/7', dure='41/3', terre_battue='11/3', salle='5/0', carpet='0/0', gazon='9/1', acryl='0/0'),
 Statistiques(annee='2023', sommaire='66/18', dure='29/7', terre_battue='8/4', salle='21/4', carpet='0/0', gazon='8/3', acryl='0/0'),
 Statistiques(annee='2022', sommaire='47/17', dure='21/7', terre_battue='15/4', salle='7/4', carpet='0/0', gazon='4/2', acryl='0/0'),
 Statistiques(annee='2021', sommaire='50/24', dure='23/10', terre_battue='10/6', salle='17/6', carpet='0/0', gazon='0/2', acryl='0/0'),
 Statistiques(annee='2020', sommaire='24/16', dure='2/6', terre_battue='10/5', salle='12/5', carpet='0/0', gazon='0/0', acryl='0/0'),
 Statistiques(annee='2019', sommaire='62/24', dure='12/6', terre_battue='17/8', salle='31/7', carpet='0/0', gazon='2/3', acryl='0/0'),
 Statistiques(annee='2018', sommaire='31/23', dure='13/9', terre_battue='17/12', salle='1/1', carpet='0/1', gazon='0/0', acryl='0/0'),
 Statistiques(annee='2017', sommaire='1/2', dure='0/0', ter

## Matchs

In [42]:
type(derniers_match)

bs4.element.Tag

# Récupération des stats de tous les joueurs

In [6]:
def get_page_joueur(liste_lien_joueur):
    for lien in liste_lien_joueur:
        reponse = get(lien)
        print(lien, reponse.status_code)
        assert reponse.status_code == 200
        time.sleep(2)

In [8]:
get_page_joueur(liens_joueurs)

https://www.tennisendirect.net/atp/jannik-sinner/ 200


KeyboardInterrupt: 